# **`Action Recognition Code On UCF11 Dataset`**

## Import necessary libraries

In [1]:
import os
import numpy as np
import cv2
import random
import matplotlib.pyplot as plt
import h5py

import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import *
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.utils import to_categorical, plot_model

from sklearn.model_selection import train_test_split

from time import time

from sklearn.metrics import classification_report

from google.colab import drive


## Retrieving UCF11 dataset from google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Set the constant seed num for reproduceability

In [ ]:
constant_seed = 20
random.seed(constant_seed)
np.random.seed(constant_seed)
tf.random.set_seed(constant_seed)

## Unrar the dataset

In [ ]:
# Don't show the output, It's not necessary.
%%capture

!pip install unrar
!unrar x "/content/drive/MyDrive/Cyshield/UCF11_updated_mpg.rar" "/content/"

## Set the height, weight, and sequence length of the input videos

In [ ]:
# Length of desired frame seq
cons_frames= 25

# Desired width and hight of each frame
cons_width, cons_hieght= 128, 128

# Select sub-labels from UCF-11 dataset
ucf11= "/content/UCF11_updated_mpg"

label_names= ['volleyball_spiking', 'basketball', 'trampoline_jumping', 'diving'] # os.listdir(f"{ucf11}")
print(label_names)

# All labels:
# ['horse_riding', 'soccer_juggling', 'biking', 'volleyball_spiking', 'tennis_swing', 'walking', 'diving', 'trampoline_jumping', 'basketball', 'golf_swing', 'swing']


['volleyball_spiking', 'basketball', 'trampoline_jumping', 'diving']


## Extract the 25 frames for each video in dataset

In [ ]:
def frames_extraction(video_path):
    '''
    This function will extract the required frames from a video after resizing and normalizing them.

    Args: video_path: The path of the video in the disk, whose frames are to be extracted.

    Returns: frames_list--> A list containing the resized and normalized frames of the video.
    '''

    # Declare a list to store video frames.
    frames_list = []

    # Read the Video File using the VideoCapture object.
    video_reader = cv2.VideoCapture(video_path)

    # Get the total number of frames in the video.
    video_frames_count = int(video_reader.get(cv2.CAP_PROP_FRAME_COUNT))

    # Calculate the the interval after which frames will be added to the list.
    skip_frames_window = max(int(video_frames_count / cons_frames), 1)

    # Iterate through the Video Frames.
    for frame_counter in range(cons_frames):

        # Set the current frame position of the video.
        video_reader.set(cv2.CAP_PROP_POS_FRAMES, frame_counter * skip_frames_window)

        # Reading the frame from the video.
        success, frame = video_reader.read()

        # Check if Video frame is not successfully read then break the loop
        if not success:
            continue

        # Resize the Frame to fixed height and width.
        resized_frame = cv2.resize(frame, (cons_hieght, cons_width))

        # Normalize the resized frame by dividing it with 255 so that each pixel value then lies between 0 and 1
        normalized_frame = resized_frame / 255

        # Append the normalized frame into the frames list
        frames_list.append(normalized_frame)

    # Release the VideoCapture object.
    video_reader.release()

    # Return the frames list.
    return frames_list

## Define a function to create the dataset and convert it into a numpy array

In [ ]:
def create_dataset():
    '''
    This function will read the data ind convert it into array d-type.
    Returns:  features:          A list containing the extracted frames of the videos.
              labels:            A list containing the indexes of the classes associated with the videos.
    '''

    # Declared Empty Lists to store the features, labels and video file path values.
    features = []
    labels = []

    # Iterating through selected classes mentioned in the label_names list
    for class_index, class_name in enumerate(label_names):

        # Display the name of the class whose data is being extracted.
        print(f'Extracting Data of Class: {class_name}')

        # Get the list of video files present in the specific class name directory.
        files_list = os.listdir(os.path.join(ucf11, class_name))

        # Iterate through all the files present in the files list.
        for file_name in files_list:

            if file_name == "Annotation" :
              continue

            for video_name in os.listdir(f"{ucf11}/{class_name}/{file_name}"):
              # Get the complete video path.
              video_file_path = os.path.join(ucf11, class_name, file_name, video_name)

              # Extract the frames of the video file.
              frames = frames_extraction(video_file_path)

              # Check if the extracted frames are equal to the cons_frames specified above.
              # So ignore the vides having frames less than the cons_frames.
              if len(frames) == cons_frames:

                  # Append the data to their repective lists.
                  features.append(frames)
                  labels.append(class_index)

    # Converting the list to numpy arrays
    features = np.asarray(features)
    labels = np.array(labels)

    # Return the frames, class index, and video file path.
    return features, labels

## It's time to create the dataset

In [ ]:
# Create the dataset
features, labels = create_dataset()
print("data is prepared for splitting")

Extracting Data of Class: volleyball_spiking
Extracting Data of Class: basketball
Extracting Data of Class: trampoline_jumping
Extracting Data of Class: diving
data is prepared for splitting


In [ ]:
print(f"the shape of features: {features.shape}")

the size of features: (529, 25, 128, 128, 3)


## Save the features and labels to prevent the lost variables while exceeding the runtime limitation

In [ ]:
np.save('labels_test.npy', labels)
with h5py.File('features_test.h5', 'w') as hf:
    hf.create_dataset('features_test', data=features, compression='gzip', compression_opts=9)

## Load features and labels after losing variables

In [ ]:
with h5py.File('/content/drive/MyDrive/Cyshield/features.h5', 'r') as hf:
    features = hf['features'][:]
print(f"ensuring the features shape: {features.shape}")

labels= np.load("/content/drive/MyDrive/Cyshield/labels.npy")
print(f"ensuring the labels shape:{labels.shape}")

ensuring the features shape: (529, 25, 128, 128, 3)
ensuring the labels shape:(529,)


## Convert categorical labels to one-hot encoding to completely separate between them

In [ ]:
one_hot_labels= to_categorical(labels)
print(one_hot_labels.shape)

(529, 4)


## Splitting data into training 90% (80% training and 10% tvalidation)![s.mp4](...) and testing 10%


In [ ]:
# Split with stratify the Data into Train ( 90% ) and Test Set ( 10% ).
features_train, features_test, labels_train, labels_test = train_test_split(features, one_hot_labels,
                                                                            test_size = 0.10, shuffle = True,
                                                                            stratify= list(labels),
                                                                            random_state = constant_seed)

## Construct the model

> The "MobileNetV2" pre-trained model utilized as a base model.

> "TimeDistributed" layer has been used for keep saving temporal information of video frames.

> LSTM-RNN layer has been used to deel with the sequence data after flattening.









In [ ]:
base_model= MobileNetV2(input_shape=(cons_hieght, cons_width, 3),
                        include_top=False,
                        weights= "imagenet")
base_model.trainable= False
#conv_model.summary()

# Build the desired model architicture
model= Sequential()
model.add(TimeDistributed(base_model, input_shape=(cons_frames, cons_hieght, cons_width, 3)))
model.add(TimeDistributed(Dropout(0.25), name="drop_out"))
model.add(TimeDistributed(Flatten(), name= 'flatten'))
model.add(LSTM(32))
model.add(Dense(len(label_names), activation= 'softmax', name="output"))

model.summary()

9406464/9406464 [==============================] - 0s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed (TimeDist  (None, 25, 4, 4, 1280)    2257984   
 ributed)                                                        
                                                                 
 drop_out (TimeDistributed)  (None, 25, 4, 4, 1280)    0         
                                                                 
 flatten (TimeDistributed)   (None, 25, 20480)         0         
                                                                 
 lstm (LSTM)                 (None, 32)                2625664   
                                                                 
 output (Dense)              (None, 4)                 132       
                                                                 
Total params: 4883780 (18.63 MB)
Trainable params: 2625796 

## Compile & train the model

In [ ]:
# Create an Instance of Early Stopping Callback.
early_stopping_callback = EarlyStopping(monitor = 'val_loss', patience = 15, mode = 'min', restore_best_weights = True)

# Save the model weights periodically during training.
model_checkpoint_callback = ModelCheckpoint(
    filepath='model_checkpoint.h5',
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True,  # Save only if the validation loss improves
)

# Compile the model and specify loss function, optimizer and metrics to the model.
model.compile(loss = 'categorical_crossentropy', optimizer = 'Adam', metrics = ["accuracy"])

# Set the callbacks
callbacks = [model_checkpoint_callback, early_stopping_callback]

# Start training the model.
model_training_history = model.fit(x = features_train, y = labels_train, epochs = 50, batch_size = 8 ,
                                             shuffle = True, validation_split = 0.15, callbacks=callbacks)

## Evaluate the model afer training, then save it

In [ ]:
# Evaluate the trained model.
model_evaluation_history = model.evaluate(features_test, labels_test)

# Save the model
model.save(f"model with {model_evaluation_history[0]}% loss and {model_evaluation_history[1]}% accuracy.h5")

## Evaluate the model on testing data

In [ ]:
# Evaluate the trained model.
model_evaluation_history = model.predict(features_test)
y_hat = np.argmax(model_evaluation_history,1)
y_hat= to_categorical(y_hat)
print(classification_report(labels_test, y_hat))
#results = label_names[index]
#model.save(f"model with {model_evaluation_history[0]}% loss and {model_evaluation_history[1]}% accuracy.h5")

## Create a function that plots the Loss/Accuracy graph for training and validation data

In [ ]:
def plot_metric(model_training_history, metric_name_1, metric_name_2, plot_name):
    '''
    This function will plot the metrics passed to it in a graph.
    Args:
        model_training_history: A history object containing a record of training and validation
                                loss values and metrics values at successive epochs
        metric_name_1:          The name of the first metric that needs to be plotted in the graph.
        metric_name_2:          The name of the second metric that needs to be plotted in the graph.
        plot_name:              The title of the graph.
    '''

    # Get metric values using metric names as identifiers.
    metric_value_1 = model_training_history.history[metric_name_1]
    metric_value_2 = model_training_history.history[metric_name_2]

    # Construct a range object which will be used as x-axis (horizontal plane) of the graph.
    epochs = range(len(metric_value_1))

    # Plot the Graph.
    plt.plot(epochs, metric_value_1, 'blue', label = metric_name_1)
    plt.plot(epochs, metric_value_2, 'red', label = metric_name_2)

    # Add title to the plot.
    plt.title(str(plot_name))

    # Add legend to the plot.
    plt.legend()

## Plot the loss metric

In [ ]:
# Visualize the training and validation loss metrices.
plot_metric(model_training_history, 'loss', 'val_loss', 'Total Loss vs Total Validation Loss')

## Plot the accuracy metric

In [ ]:
# Visualize the training and validation accuracy metrices.
plot_metric(model_training_history, 'accuracy', 'val_accuracy', 'Total Accuracy vs Total Validation Accuracy')

## Loading the model * (it trained on kaggle notebook)

In [ ]:
model = load_model("/content/drive/MyDrive/Cyshield/model_with_0.0419%_loss_and_0.9811%_accuracy.h5")

## Create a function to recognize separate actions along video

In [ ]:
def predict_single_action(video_file_path, output_file_path, cons_frames):
    '''
    This function will perform single action recognition prediction on a video using the LRCN model.
    Args:
    video_file_path:  The path of the video stored in the disk on which the action recognition is to be performed.
    SEQUENCE_LENGTH:  The fixed number of frames of a video that can be passed to the model as one sequence.
    '''

    # Initialize the VideoCapture object to read from the video file.
    video_reader = cv2.VideoCapture(video_file_path)

    # Get the width and height of the video.
    original_video_width = int(video_reader.get(cv2.CAP_PROP_FRAME_WIDTH))
    original_video_height = int(video_reader.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Initialize the VideoWriter Object to store the output video in the disk.
    video_writer = cv2.VideoWriter(output_file_path, cv2.VideoWriter_fourcc('M', 'P', '4', 'V'),
                                   video_reader.get(cv2.CAP_PROP_FPS), (original_video_width, original_video_height))
    # Declare a list to store video frames we will extract.
    frames_list = []

    # Initialize a variable to store the predicted action being performed in the video.
    predicted_class_name = ''

    # Get the number of frames in the video.
    video_frames_count = int(video_reader.get(cv2.CAP_PROP_FRAME_COUNT))

    # Calculate the interval after which frames will be added to the list.
    skip_frames_window = max(int(video_frames_count/cons_frames),1)

    # Iterating the number of times equal to the fixed length of sequence.
    for frame_counter in range(cons_frames):

        # Set the current frame position of the video.
        video_reader.set(cv2.CAP_PROP_POS_FRAMES, frame_counter * skip_frames_window)

        # Read a frame.
        success, frame = video_reader.read()

        # Check if frame is not read properly then break the loop.
        if not success:
            break

        # Resize the Frame to fixed Dimensions.
        resized_frame = cv2.resize(frame, (cons_hieght, cons_width))

        # Normalize the resized frame by dividing it with 255 so that each pixel value then lies between 0 and 1.
        normalized_frame = resized_frame / 255

        # Appending the pre-processed frame into the frames list
        frames_list.append(normalized_frame)

    # Pass the normalized frames to the model and get the predicted probabilities.
    predicted_labels_probabilities = model.predict(np.expand_dims(frames_list, axis = 0))[0]

    # Get the index of class with highest probability.
    predicted_label = np.argmax(predicted_labels_probabilities)

    # Get the class name using the retrieved index.
    predicted_class_name = label_names[predicted_label]

    # Release the VideoCapture object
    video_reader.release()

    # Recreate VideoCapture for generating labels
    video_reader = cv2.VideoCapture(video_file_path)

    for label in range(int(video_reader.get(cv2.CAP_PROP_FRAME_COUNT))):
        # Set the frame based on its position
        video_reader.set(cv2.CAP_PROP_POS_FRAMES, label)

        # Read the frame.
        ok, frame = video_reader.read()

        # Check if frame is not read properly then break the loop.
        if not ok:
            break

        # Write predicted class name on top of the frame.
        cv2.putText(frame, predicted_class_name, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        # Write The frame into the disk using the VideoWriter Object.
        video_writer.write(frame)

    # Release VideoCapture and VideoWriter objects.
    video_reader.release()
    video_writer.release()

In [ ]:
# Testing random downloaded video
test_path= "/content/trampoline.mp4"

# Construct the output video path.
output_video_file_path = 'trampoline_jumping_test.mp4'

# Perform Action Recognition on the Test Video.
predict_single_action(test_path, output_video_file_path, cons_frames)


1/1 [==============================] - 1s 957ms/step
